# Lab 02: Iocane Powder

## Overview

> Man in Black: All right. Where is the poison? The battle of wits has begun. It ends when you decide and we both drink, and find out who is right... and who is dead. 

The line above is from the perennial favorite 1980s movie adaptation of William Goldman's *The Princess Bride*, wherein a mysterious hero sits down to a battle of wits with a villainous Sicilian kidnapper. The setup: two cups positioned between the two, one of which (purportedly) contains a colorless, odorless, lethal poison (viz., iocane powder). After a guess is made as to which cup contains the poison, both drink, and the winner is the one left standing.

For this machine problem you will write a program that simulates multiple rounds of this battle of wits, allowing the player to repeatedly guess which cup is poisoned. The computer will "place" the poison before the player guesses, and
will reveal who is right... and who is dead, afterwards.

At the outset, the computer will always place the poison in cup 2 before letting the player guess, but after enough guesses have been entered the computer will start to place the poison based on the pattern of previous guesses so as to outsmart the player.

Here's a sample game session (note how the silly player keeps alternating guesses, and that the computer catches on to this fact after a while):

    Where is the iocane powder: my cup (1) or yours (2)? 1
    Wrong! Ha! Never bet against a Sicilian!

    You died 1 times, and I drank the poison 0 times

    Where is the iocane powder: my cup (1) or yours (2)? 2
    Good guess! Ack! I drank the poison!

    You died 1 times, and I drank the poison 1 times

    Where is the iocane powder: my cup (1) or yours (2)? 1
    Wrong! Ha! Never bet against a Sicilian!

    You died 2 times, and I drank the poison 1 times

    Where is the iocane powder: my cup (1) or yours (2)? 2
    Good guess! Ack! I drank the poison!

    You died 2 times, and I drank the poison 2 times

    Where is the iocane powder: my cup (1) or yours (2)? 1
    Wrong! Ha! Never bet against a Sicilian!

    You died 3 times, and I drank the poison 2 times

    Where is the iocane powder: my cup (1) or yours (2)? 2
    Wrong! Ha! Never bet against a Sicilian!

    You died 4 times, and I drank the poison 2 times

    Where is the iocane powder: my cup (1) or yours (2)? 1
    Wrong! Ha! Never bet against a Sicilian!

    You died 5 times, and I drank the poison 2 times

    Where is the iocane powder: my cup (1) or yours (2)? 2
    Wrong! Ha! Never bet against a Sicilian!

    You died 6 times, and I drank the poison 2 times

    Where is the iocane powder: my cup (1) or yours (2)? 1
    Wrong! Ha! Never bet against a Sicilian!

    You died 7 times, and I drank the poison 2 times


## Implementation

To keep track of the pattern of previous guesses, you will use a dictionary that maps a pattern (of fixed length) to a list of counts for the subsequent guess. 

For instance, imagine that the computer observes the player continuing to alternate guesses across ten separate attempts, like so: '1', '2', '1', '2', '1', '2', '1', '2', '1', '2'. If we are using a pattern detection length of three, then after the fourth guess we can create an entry in our dictionary that maps the key '121' to the list [0, 1], where the second value (1) in the list indicates that the player guessed '2' following the sequence '1', '2', '1'. After the fifth guess, we create the entry '212' &rarr; [1, 0], and after the sixth guess we update the value for '121' to [0, 2] (since the user guesses '2' again, after the sequence '1', '2', '1').

Once the player enters a series of guesses that matches a previously seen pattern, the computer should place the poison in the cup that the player is *least likely to guess next*. When the player enters the next guess, the dictionary should be updated to reflect the actual guess.

This means that if the computer has yet to see a given pattern of guesses, or when the counts are tied, it will have to place the poison "blindly" --- your implementation should simply place the poison furthest away from itself (cup 2). 

### `record_guess`

The first function you are to complete is `record_guess`. It will take the following arguments:

- a dictionary to update (possibly containing previously recorded pattern &rarr; list mappings)
- a pattern string
- a guess -- which is either '1' or '2'.  

If necessary, the function will create a new entry for the pattern (if one doesn't already exist), then record the updated count for the guess. Since the dictionary is updated in place (i.e., mutated), the function will not return anything. 

Complete the function below, checking your implementation with the test cases that follow when you're ready. Note that in the future, the bulk of the description for functions we ask you to implement will simply be placed in the functions' docstrings, as below.

*Hints: the [`int`](https://docs.python.org/3/library/functions.html#int) function can be used to convert strings to integers, and you might find the dictionary's [`setdefault`](https://docs.python.org/3/library/stdtypes.html?highlight=setdefault#dict.setdefault) method useful.*

In [1]:
def record_guess(pattern_dict, pattern, guess):
    """Updates the `pattern_dict` dictionary by either creating a new entry
    or updating an existing entry for key `pattern`, increasing the count 
    corresponding to `guess` in the list."""
    if not pattern in pattern_dict:
        pattern_dict[pattern] = [0, 0]
        
    if guess == '1':
        pattern_dict[pattern][0] += 1
    elif guess == '2':
        pattern_dict[pattern][1] += 1
        
    

In [2]:
# (5 points)

from unittest import TestCase
tc = TestCase()
d = {}
record_guess(d, '121', '1')
tc.assertDictEqual(d, {'121': [1, 0]})
record_guess(d, '222', '2')
record_guess(d, '121', '1')
tc.assertDictEqual(d, {'121': [2, 0], '222': [0, 1]})
record_guess(d, '122', '2')
record_guess(d, '121', '2')
record_guess(d, '222', '2')
tc.assertDictEqual(d, {'121': [2, 1], '122': [0, 1], '222': [0, 2]})

### `next_placement`

The next function you'll write will take a dictionary of pattern &rarr; counts mappings and a string representing the pattern of most recent guesses, and return the next best location (either '1' or '2') for the poison (i.e., to try and outwit the player). If the pattern hasn't been seen previously or the counts are tied, the function should return '2'.

In [3]:
def next_placement(pattern_dict, pattern):
    if not pattern in pattern_dict:
        return '2'
    elif int(pattern_dict[pattern][0]) >= int(pattern_dict[pattern][1]):
        return '2'
    else:
        return '1'
            

In [4]:
# (5 points)

from unittest import TestCase
tc = TestCase()
tc.assertEqual(next_placement({}, '121'), '2')
tc.assertEqual(next_placement({'121': [2, 0]}, '121'), '2')
tc.assertEqual(next_placement({'121': [2, 5]}, '121'), '1')
tc.assertEqual(next_placement({'121': [2, 5]}, '212'), '2')
tc.assertEqual(next_placement({'121': [5, 5]}, '121'), '2')
tc.assertEqual(next_placement({'121': [15, 5]}, '121'), '2')
tc.assertEqual(next_placement({'121': [2, 5],
                               '212': [1, 1]}, '212'), '2')
tc.assertEqual(next_placement({'121': [2, 5],
                               '212': [1, 3]}, '212'), '1')

### `play_interactive`

Now for the fun bit. The function `play_interactive` will take just one argument --- the length of patterns to use as keys in the dictionary --- and will start an interactive game session, reading either '1' or '2' from the player as guesses, using the functions you wrote above and producing output as shown in the sample game session at the beginning of this writeup. If the player types in any other input (besides '1' or '2'), the game should terminate.

*Hint: the [`input`](https://docs.python.org/3/library/functions.html#input) function can be used to read input from the user as a string.*

In [5]:
def play_interactive(pattern_length=4):
    nums = [""]*pattern_length
    right = 0
    wrong = 0
    pats = {}
    comp_guess = "0"
    while True:
        guess = input("Where is the iocane powder: my cup (1) or yours (2)?")
        if guess != '1' and guess != '2':
            break
        if right + wrong < pattern_length:
            nums[right + wrong] = guess
        if right + wrong < pattern_length or comp_guess == "2":
            if guess == "1":
                wrong += 1
                print ("Wrong! Ha! Never bet against a Sicilian!")
            else:
                right += 1
                print("Good guess! Ack! I drank the poison!")
            print ("\n") 
            print ("You died " + str(wrong) + " times, and I drank the poison " + str(right) + " times")
            print ("\n")    
        if comp_guess == "1":
            if guess == "2":
                wrong += 1
                print ("Wrong! Ha! Never bet against a Sicilian!")
            else:
                right += 1
                print("Good guess! Ack! I drank the poison!")
            print ("\n") 
            print ("You died " + str(wrong) + " times, and I drank the poison " + str(right) + " times")
            print ("\n") 
        if right + wrong > pattern_length:
            temp = [""]*pattern_length
            for i in range(3):
                temp[i] = nums[i+1]
            temp[3] = guess
            nums = temp
        if right + wrong >= pattern_length:
            pattern = ""
            for i in nums:
                pattern += i
            record_guess(pats, pattern, guess)
            comp_guess = next_placement(pats, pattern)
            
            
play_interactive()        

Where is the iocane powder: my cup (1) or yours (2)?2
Good guess! Ack! I drank the poison!


You died 0 times, and I drank the poison 1 times


Where is the iocane powder: my cup (1) or yours (2)?1
Wrong! Ha! Never bet against a Sicilian!


You died 1 times, and I drank the poison 1 times


Where is the iocane powder: my cup (1) or yours (2)?2
Good guess! Ack! I drank the poison!


You died 1 times, and I drank the poison 2 times


Where is the iocane powder: my cup (1) or yours (2)?


## `play_batch`

Finally, so that we can check your implementation against a lengthier sequence of guesses without having to play an interactive session, implement the `play_batch` function, which will take the `pattern_length` argument as your `play_interactive` function did, but will also take a sequence of guesses. The function will return the total numbers of wins and losses, as determined by the same algorithm as before.

In [72]:
def play_batch(guesses, pattern_length=4):
    nums = [""]*pattern_length
    right = 0
    wrong = 0
    pats = {}
    tracker = 0
    comp_guess = "2"
    plays = 0
    for i in guesses:
        pattern = ""
        guess = i
        if comp_guess == "2":
            if guess == "1":
                wrong += 1
            elif guess == "2":
                right += 1
        if comp_guess == "1":
            if guess == "1":
                right += 1
            elif guess == "2":
                wrong += 1
        tracker += 1
        if tracker <= pattern_length:
            nums[tracker - 1] = guess
        if tracker > pattern_length:
            for i in range (pattern_length):
                pattern = pattern + nums[i]
            record_guess(pats, pattern, guess)
            temp = [""]*pattern_length
            for i in range (pattern_length - 1):
                temp[i] = nums[i + 1]
            temp[pattern_length - 1] = guess
            nums = temp
            pattern = ""
            for i in range (pattern_length):
                pattern = pattern + nums[i]
            comp_guess = next_placement(pats, pattern)
        #print (right, wrong)
    return(right, wrong) 

In [73]:
# (10 points)

from unittest import TestCase
tc = TestCase()
tc.assertEqual(play_batch(['1', '1', '1', '1', '1', '1'], 3), (0, 6))
tc.assertEqual(play_batch(['1', '2', '1', '2', '1', '2'], 3), (2, 4))
tc.assertEqual(play_batch(['1', '2', '1', '2', '1', '2'], 4), (3, 3))
tc.assertEqual(play_batch(['1', '2'] * 100, 5), (3, 197))
tc.assertEqual(play_batch(['1', '1', '2', '1', '2', '1'] * 100, 2), (398, 202))
tc.assertEqual(play_batch(['1', '1', '2', '1', '2', '1'] * 100, 3), (201, 399))
tc.assertEqual(play_batch(['1', '1', '2', '1', '2', '1'] * 100, 5), (4, 596))
import random
random.seed(0, version=2)
tc.assertEqual(play_batch((random.choice(['1', '2']) for _ in range(10000)), 4), (5047, 4953))

0 1
0 2
0 3
0 4
0 5
0 6
0 1
1 1
1 2
2 2
2 3
2 4
0 1
1 1
1 2
2 2
2 3
3 3
0 1
1 1
1 2
2 2
2 3
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
3 33
3 34
3 35
3 36
3 37
3 38
3 39
3 40
3 41
3 42
3 43
3 44
3 45
3 46
3 47
3 48
3 49
3 50
3 51
3 52
3 53
3 54
3 55
3 56
3 57
3 58
3 59
3 60
3 61
3 62
3 63
3 64
3 65
3 66
3 67
3 68
3 69
3 70
3 71
3 72
3 73
3 74
3 75
3 76
3 77
3 78
3 79
3 80
3 81
3 82
3 83
3 84
3 85
3 86
3 87
3 88
3 89
3 90
3 91
3 92
3 93
3 94
3 95
3 96
3 97
3 98
3 99
3 100
3 101
3 102
3 103
3 104
3 105
3 106
3 107
3 108
3 109
3 110
3 111
3 112
3 113
3 114
3 115
3 116
3 117
3 118
3 119
3 120
3 121
3 122
3 123
3 124
3 125
3 126
3 127
3 128
3 129
3 130
3 131
3 132
3 133
3 134
3 135
3 136
3 137
3 138
3 139
3 140
3 141
3 142
3 143
3 144
3 145
3 146
3 147
3 148
3 149
3 150
3 151
3 152
3 153
3 154
3 155
3 156
3 157
3 158
3 159
3 160
3 161
3 162
3 163
3 164
3 165
3 166
3 167
3 168
3 169
3 170
3 17

4 219
4 220
4 221
4 222
4 223
4 224
4 225
4 226
4 227
4 228
4 229
4 230
4 231
4 232
4 233
4 234
4 235
4 236
4 237
4 238
4 239
4 240
4 241
4 242
4 243
4 244
4 245
4 246
4 247
4 248
4 249
4 250
4 251
4 252
4 253
4 254
4 255
4 256
4 257
4 258
4 259
4 260
4 261
4 262
4 263
4 264
4 265
4 266
4 267
4 268
4 269
4 270
4 271
4 272
4 273
4 274
4 275
4 276
4 277
4 278
4 279
4 280
4 281
4 282
4 283
4 284
4 285
4 286
4 287
4 288
4 289
4 290
4 291
4 292
4 293
4 294
4 295
4 296
4 297
4 298
4 299
4 300
4 301
4 302
4 303
4 304
4 305
4 306
4 307
4 308
4 309
4 310
4 311
4 312
4 313
4 314
4 315
4 316
4 317
4 318
4 319
4 320
4 321
4 322
4 323
4 324
4 325
4 326
4 327
4 328
4 329
4 330
4 331
4 332
4 333
4 334
4 335
4 336
4 337
4 338
4 339
4 340
4 341
4 342
4 343
4 344
4 345
4 346
4 347
4 348
4 349
4 350
4 351
4 352
4 353
4 354
4 355
4 356
4 357
4 358
4 359
4 360
4 361
4 362
4 363
4 364
4 365
4 366
4 367
4 368
4 369
4 370
4 371
4 372
4 373
4 374
4 375
4 376
4 377
4 378
4 379
4 380
4 381
4 382
4 383
4 384
4 38

628 591
629 591
629 592
629 593
630 593
631 593
632 593
632 594
632 595
632 596
633 596
634 596
635 596
636 596
637 596
637 597
638 597
638 598
638 599
639 599
640 599
640 600
640 601
641 601
641 602
641 603
642 603
642 604
643 604
644 604
645 604
645 605
645 606
645 607
646 607
647 607
647 608
648 608
649 608
649 609
649 610
650 610
650 611
650 612
651 612
652 612
653 612
654 612
654 613
654 614
654 615
655 615
656 615
657 615
657 616
657 617
658 617
659 617
659 618
659 619
660 619
660 620
661 620
662 620
663 620
663 621
663 622
664 622
665 622
666 622
666 623
667 623
667 624
668 624
669 624
670 624
670 625
671 625
672 625
672 626
672 627
673 627
674 627
674 628
675 628
675 629
676 629
676 630
677 630
677 631
678 631
679 631
679 632
679 633
679 634
680 634
680 635
680 636
680 637
680 638
680 639
681 639
682 639
683 639
683 640
684 640
684 641
684 642
684 643
685 643
685 644
685 645
685 646
686 646
687 646
687 647
688 647
688 648
689 648
689 649
689 650
689 651
690 651
691 651
692 651


1415 1346
1415 1347
1416 1347
1417 1347
1418 1347
1418 1348
1419 1348
1420 1348
1421 1348
1422 1348
1423 1348
1424 1348
1424 1349
1425 1349
1425 1350
1426 1350
1427 1350
1427 1351
1427 1352
1427 1353
1427 1354
1428 1354
1429 1354
1429 1355
1430 1355
1431 1355
1431 1356
1432 1356
1432 1357
1432 1358
1433 1358
1434 1358
1435 1358
1435 1359
1435 1360
1436 1360
1437 1360
1438 1360
1438 1361
1438 1362
1438 1363
1438 1364
1439 1364
1439 1365
1440 1365
1441 1365
1441 1366
1441 1367
1441 1368
1441 1369
1441 1370
1441 1371
1441 1372
1442 1372
1443 1372
1444 1372
1445 1372
1446 1372
1447 1372
1448 1372
1449 1372
1449 1373
1450 1373
1451 1373
1452 1373
1453 1373
1453 1374
1453 1375
1453 1376
1454 1376
1455 1376
1455 1377
1456 1377
1456 1378
1457 1378
1457 1379
1457 1380
1458 1380
1459 1380
1459 1381
1460 1381
1460 1382
1461 1382
1461 1383
1461 1384
1461 1385
1462 1385
1462 1386
1463 1386
1464 1386
1464 1387
1465 1387
1466 1387
1467 1387
1467 1388
1467 1389
1467 1390
1468 1390
1468 1391
1469 1391


2181 2039
2181 2040
2182 2040
2183 2040
2184 2040
2185 2040
2186 2040
2186 2041
2186 2042
2186 2043
2187 2043
2187 2044
2188 2044
2189 2044
2190 2044
2190 2045
2190 2046
2190 2047
2190 2048
2191 2048
2191 2049
2191 2050
2192 2050
2192 2051
2192 2052
2193 2052
2193 2053
2193 2054
2194 2054
2194 2055
2195 2055
2196 2055
2197 2055
2197 2056
2197 2057
2198 2057
2198 2058
2198 2059
2198 2060
2198 2061
2198 2062
2198 2063
2199 2063
2200 2063
2200 2064
2201 2064
2202 2064
2202 2065
2202 2066
2202 2067
2203 2067
2204 2067
2205 2067
2205 2068
2205 2069
2206 2069
2207 2069
2208 2069
2208 2070
2208 2071
2208 2072
2209 2072
2209 2073
2210 2073
2210 2074
2211 2074
2211 2075
2211 2076
2211 2077
2212 2077
2213 2077
2214 2077
2215 2077
2216 2077
2216 2078
2216 2079
2216 2080
2217 2080
2217 2081
2217 2082
2217 2083
2218 2083
2219 2083
2220 2083
2220 2084
2220 2085
2221 2085
2222 2085
2223 2085
2223 2086
2224 2086
2224 2087
2224 2088
2224 2089
2225 2089
2226 2089
2226 2090
2227 2090
2227 2091
2227 2092


2654 2565
2655 2565
2655 2566
2655 2567
2655 2568
2656 2568
2656 2569
2657 2569
2657 2570
2658 2570
2659 2570
2659 2571
2660 2571
2661 2571
2662 2571
2663 2571
2663 2572
2663 2573
2663 2574
2663 2575
2663 2576
2664 2576
2664 2577
2665 2577
2666 2577
2667 2577
2668 2577
2668 2578
2668 2579
2668 2580
2668 2581
2669 2581
2669 2582
2670 2582
2671 2582
2671 2583
2672 2583
2672 2584
2673 2584
2673 2585
2674 2585
2674 2586
2674 2587
2675 2587
2675 2588
2676 2588
2677 2588
2677 2589
2677 2590
2677 2591
2678 2591
2679 2591
2680 2591
2681 2591
2681 2592
2682 2592
2682 2593
2682 2594
2682 2595
2682 2596
2683 2596
2683 2597
2683 2598
2683 2599
2684 2599
2684 2600
2684 2601
2685 2601
2686 2601
2686 2602
2686 2603
2687 2603
2687 2604
2688 2604
2689 2604
2689 2605
2689 2606
2690 2606
2690 2607
2690 2608
2691 2608
2692 2608
2692 2609
2692 2610
2692 2611
2693 2611
2693 2612
2694 2612
2694 2613
2694 2614
2694 2615
2694 2616
2694 2617
2694 2618
2695 2618
2695 2619
2695 2620
2695 2621
2696 2621
2696 2622


3423 3296
3424 3296
3424 3297
3425 3297
3425 3298
3425 3299
3426 3299
3426 3300
3426 3301
3427 3301
3428 3301
3429 3301
3429 3302
3430 3302
3430 3303
3431 3303
3432 3303
3433 3303
3434 3303
3434 3304
3435 3304
3435 3305
3436 3305
3436 3306
3437 3306
3437 3307
3437 3308
3437 3309
3438 3309
3439 3309
3440 3309
3441 3309
3442 3309
3443 3309
3444 3309
3445 3309
3446 3309
3446 3310
3446 3311
3446 3312
3447 3312
3447 3313
3447 3314
3447 3315
3447 3316
3447 3317
3447 3318
3448 3318
3448 3319
3449 3319
3449 3320
3450 3320
3450 3321
3451 3321
3451 3322
3452 3322
3453 3322
3453 3323
3454 3323
3454 3324
3455 3324
3455 3325
3455 3326
3455 3327
3456 3327
3456 3328
3456 3329
3456 3330
3457 3330
3458 3330
3459 3330
3460 3330
3460 3331
3460 3332
3461 3332
3462 3332
3463 3332
3463 3333
3463 3334
3463 3335
3464 3335
3465 3335
3465 3336
3466 3336
3466 3337
3467 3337
3468 3337
3468 3338
3469 3338
3469 3339
3469 3340
3470 3340
3471 3340
3472 3340
3472 3341
3472 3342
3473 3342
3474 3342
3474 3343
3474 3344


3900 3806
3901 3806
3901 3807
3902 3807
3903 3807
3903 3808
3904 3808
3904 3809
3904 3810
3905 3810
3906 3810
3906 3811
3906 3812
3906 3813
3906 3814
3906 3815
3906 3816
3906 3817
3906 3818
3907 3818
3907 3819
3908 3819
3908 3820
3908 3821
3908 3822
3909 3822
3910 3822
3910 3823
3911 3823
3912 3823
3913 3823
3914 3823
3914 3824
3914 3825
3914 3826
3914 3827
3915 3827
3915 3828
3915 3829
3915 3830
3916 3830
3916 3831
3917 3831
3917 3832
3917 3833
3918 3833
3918 3834
3918 3835
3918 3836
3919 3836
3920 3836
3921 3836
3922 3836
3922 3837
3923 3837
3924 3837
3925 3837
3925 3838
3926 3838
3926 3839
3926 3840
3926 3841
3926 3842
3926 3843
3926 3844
3927 3844
3928 3844
3929 3844
3929 3845
3929 3846
3930 3846
3930 3847
3930 3848
3931 3848
3932 3848
3933 3848
3934 3848
3935 3848
3936 3848
3936 3849
3936 3850
3937 3850
3938 3850
3938 3851
3939 3851
3939 3852
3939 3853
3940 3853
3941 3853
3942 3853
3942 3854
3942 3855
3942 3856
3942 3857
3943 3857
3943 3858
3943 3859
3943 3860
3943 3861
3944 3861


4539 4429
4539 4430
4539 4431
4540 4431
4540 4432
4540 4433
4540 4434
4541 4434
4542 4434
4542 4435
4542 4436
4542 4437
4543 4437
4543 4438
4544 4438
4544 4439
4544 4440
4545 4440
4545 4441
4546 4441
4547 4441
4547 4442
4547 4443
4547 4444
4547 4445
4547 4446
4548 4446
4549 4446
4550 4446
4550 4447
4550 4448
4550 4449
4550 4450
4551 4450
4552 4450
4553 4450
4553 4451
4554 4451
4555 4451
4556 4451
4556 4452
4557 4452
4557 4453
4558 4453
4558 4454
4558 4455
4559 4455
4559 4456
4560 4456
4560 4457
4561 4457
4562 4457
4563 4457
4564 4457
4564 4458
4564 4459
4565 4459
4566 4459
4567 4459
4568 4459
4568 4460
4569 4460
4570 4460
4570 4461
4570 4462
4570 4463
4570 4464
4571 4464
4572 4464
4573 4464
4574 4464
4575 4464
4576 4464
4577 4464
4577 4465
4577 4466
4578 4466
4579 4466
4579 4467
4580 4467
4580 4468
4581 4468
4582 4468
4582 4469
4582 4470
4583 4470
4583 4471
4583 4472
4583 4473
4583 4474
4583 4475
4584 4475
4585 4475
4586 4475
4587 4475
4587 4476
4587 4477
4588 4477
4589 4477
4590 4477
